In [1]:
import os
import glob
import time
import plotly
import requests
import numpy as np
import pandas as pd

plotly.offline.init_notebook_mode(connected=True)

In [2]:
DT_FORMAT = "%Y-%m-%d"

## Setup Exam Dates

In [3]:
due_dates = pd.DataFrame(columns = ['item', 'date'],
                          data = [['ENVS 200', '2017-08-09'],
                                  ['ECE 429', '2017-07-31'],
                                  ['ECE 454', '2017-08-10'],
                                  ['ECE 457A', '2017-08-01']])
due_dates['date'] = pd.to_datetime(due_dates.date)
due_dates

,item,date
0,ENVS 200,2017-08-09
1,ECE 429,2017-07-31
2,ECE 454,2017-08-10
3,ECE 457A,2017-08-01


## Trello Integration

In [4]:
# define trello auth parameters
board = ''
applicationKey = ''
token = ''

In [5]:
# Get a list of active courses
endpoint = 'https://api.trello.com/1/board/{}/lists?cards=open&card_fields=all&key={}&token={}'.format(board, applicationKey, token)
r = requests.get(endpoint)

In [6]:
list_id_name_map = {}
card_id_task_map = {}
card_id_list_id_map = {}

for l in r.json():
    list_id_name_map[l['id']] = l['name']
    for card in l['cards']:
        card_id_task_map[card['id']] = card['name']
        card_id_list_id_map[card['id']] = l['id']
        
def getList(cardId):
    return card_id_list_id_map[cardId]

def getListName(listId):
    return list_id_name_map[listId]

In [7]:
# Get a list of all checklists
endpoint = 'https://api.trello.com/1/board/{}/checklists?key={}&token={}'.format(board, applicationKey, token)
r = requests.get(endpoint)

In [8]:
rows = []

for checklist in r.json():
    row = []
    # We only care about active tasks
    if checklist['idCard'] in card_id_task_map:
        if '[' not in checklist['name']: continue
        row.append(getListName(getList(checklist['idCard']))) # course
        row.append(card_id_task_map[checklist['idCard']]) # task
        
        score = checklist['name'].replace('[', '').replace(']', '')
        if not score.isdigit(): score = ''.join(filter(str.isdigit, score))
        score = int(score)
        
        row.append(score * len(checklist['checkItems'])) # score
        score_done = sum([1 for i in checklist['checkItems'] if i['state'] == 'complete']) * score
        row.append(score_done)
        rows.append(row)

In [9]:
df = pd.DataFrame(rows, columns=['course', 'task', 'total', 'complete'])
df.head()

,course,task,total,complete
0,ENVS 200,Chapter Notes,560,560
1,ECE 457A,Notes,280,280
2,ECE 457A,Assignments,70,0
3,ECE 429,Notes,100,100
4,ECE 429,Notes,300,300


In [10]:
# Persist today's dataframe to disk
todaysFile = time.strftime('data/%Y-%m-%d.csv')
if os.path.exists(todaysFile): os.remove(todaysFile)
if not os.path.exists(os.path.dirname(todaysFile)): os.makedirs(os.path.dirname(todaysFile))
df.to_csv(todaysFile, index=None)

## Plot!

In [11]:
df_daily = pd.DataFrame()
for csv in glob.glob('data/*.csv'):
    df_temp = pd.read_csv(csv)
    df_temp['date'] = csv.replace('data/', '').replace('.csv', '')
    df_daily = df_temp if df_daily.empty else df_daily.append(df_temp)

df_daily['date'] = pd.to_datetime(df_daily.date)
    
overall = df_daily.groupby(['date']).sum()
overall['remaining'] = overall['total'] - overall['complete']
overall

,total,complete,remaining
date,,,
2017-07-25,2785,280,2505
2017-07-26,2785,360,2425
2017-07-27,2785,485,2300
2017-07-28,2785,605,2180
2017-07-29,2785,805,1980
2017-07-30,2785,950,1835
2017-07-31,2785,1100,1685
2017-08-01,2785,1425,1360
2017-08-02,2785,1425,1360


In [12]:
overall['complete'].diff()

date
2017-07-25      NaN
2017-07-26     80.0
2017-07-27    125.0
2017-07-28    120.0
2017-07-29    200.0
2017-07-30    145.0
2017-07-31    150.0
2017-08-01    325.0
2017-08-02      0.0
2017-08-03    100.0
2017-08-04    250.0
2017-08-05      0.0
2017-08-06    175.0
2017-08-07    150.0
2017-08-08    210.0
2017-08-09    170.0
2017-08-10    135.0
2017-08-24      0.0
Name: complete, dtype: float64

In [13]:
plotly.offline.iplot({
'data': [{
    'x': overall['total'].keys(),
    'y': overall['remaining'].values
}],
'layout': {
    'title': 'Overall Burndown',
    'xaxis': {
        'range': ['2017-07-25','2017-08-10']
    },
    'yaxis': {
        'range': [0, max(overall['remaining'].values) * 1.05]
    },
    'paper_bgcolor': 'rgba(0,0,0,0)',
    'plot_bgcolor': 'rgba(0,0,0,0)'
}})

# xaxis=dict(autorange='reversed', range=[0, 10])

In [14]:
course_burndown = df_daily.groupby(['course', 'date']).sum()
course_burndown['remaining'] = course_burndown['total'] - course_burndown['complete']
course_burndown = course_burndown.reset_index()
course_burndown.head()

,course,date,total,complete,remaining
0,ECE 429,2017-07-25,660,0,660
1,ECE 429,2017-07-26,660,0,660
2,ECE 429,2017-07-27,660,85,575
3,ECE 429,2017-07-28,660,205,455
4,ECE 429,2017-07-29,660,325,335


In [15]:
data = []
colours = ['#1F77B4','#FF7F0E','#2CA02C','#D62728','#9575D2','#8C564B','#E377C0','#7F7F7F','#BCBD22','#17BECF']
i = 0
for course in course_burndown['course'].unique():
    # Grab course data
    x = course_burndown[course_burndown.course == course].date
    y = course_burndown[course_burndown.course == course].remaining
    
    # Get number of days till due
    today = course_burndown[course_burndown.course == course].date.iloc[-1]
    start = course_burndown[course_burndown.course == course].date.iloc[0]
    end = due_dates[due_dates.item == course].date.iloc[0]
    days = (end-start).days + 1
    ellapsed = (today-start).days
    
    # Fit a line to the data using last velocity
    z = np.polyfit(x=[ellapsed - 1, ellapsed], y=y.iloc[-2:], deg=1)
    p = np.poly1d(z)
    
    # Plot the course burndown (from start till exam day)
    data.append({
        'x': x[0:days],
        'y': y[0:days],
        'name': course,
        'legendgroup': course,
        'line': {
            'color': colours[i]
        }
    })
    
    # Plot the line of best fit (from today till exam day)
    data.append({
        'x': pd.date_range(today, end),
        'y': p(np.arange(ellapsed, days)),
        'name': 'Estimated',
        'legendgroup': course,
        'line': {
            'color': colours[i],
            'dash': 'dot'
        }
    })
    
    i += 1

# Create a list of vertical lines at exam dates, use plotly default colours
colours = ['#1F77B4','#FF7F0E','#2CA02C','#D62728','#9575D2','#8C564B','#E377C0','#7F7F7F','#BCBD22','#17BECF']
shapes = []
for course in course_burndown['course'].unique():
    shapes.append({
        'type': 'line',
        'x0': due_dates[due_dates.item == course].date.iloc[0].strftime(DT_FORMAT),
        'y0': 0,
        'x1': due_dates[due_dates.item == course].date.iloc[0].strftime(DT_FORMAT),
        'y1': 125,
        'line': {
            'color': colours.pop(0),
            'width': 1.5,
        }
    })

plotly.offline.iplot({
'data': data,
'layout': {
    'title': 'Burndown By Subject',
    'xaxis': {
        'range': ['2017-07-25','2017-08-10']
    },
    'yaxis': {
        'range': [0, max(course_burndown['remaining'].values) * 1.05]
    },
    'shapes': shapes,
    'paper_bgcolor': 'rgba(0,0,0,0)',
    'plot_bgcolor': 'rgba(0,0,0,0)'
}})

# xaxis=dict(autorange='reversed', range=[0, 10])